SSA없이, 가변길이 데이터로 x,y좌표 예측

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# directory/folder path
excel_dir_path = "C:/Users/user/Desktop/gait_classification/excel_files"

# list to store files
excel_files = []

# Iterate directory
for file_path in os.listdir(excel_dir_path):
    # check if current file_path is a file
    if os.path.isfile(os.path.join(excel_dir_path, file_path)):
        # add filename to list
        excel_files.append(file_path)
print(excel_files)

['DDPGtrainedAgent_3D_06_22_2024_1810_0.01_2024_06_30_1918.xlsx', 'TD3trainedAgent_06_26_2024_1840_0.01_2024_06_30_2008.xlsx']


In [3]:
from pyts.decomposition import SingularSpectrumAnalysis

In [4]:
# 1차원 배열을 SSA해서 trend와 periodic을 추출하여 리턴하는 함수
def do_SSA(lst):
    window_size = 20
    ssa = SingularSpectrumAnalysis(window_size=window_size)
    components = ssa.fit_transform(np.array(lst).reshape(1,-1))
    trend = components[0, 0, :]
    periodic = components[0, 1, :]
    noise=lst-(trend+periodic)
    return trend, periodic, noise

# SSA로 얻은 trend와 periodic에서 주파수 성분을 추출하여 단일값을 리턴하는 함수(peak)
def fourier_transform(trend, periodic, noise):
    time=np.linspace(0, 10, len(trend)) # 10초 simulation
    trend_frequency_domain = np.fft.fft(trend)
    periodic_frequency_domain = np.fft.fft(periodic)
    noise_frequency_domain = np.fft.fft(noise)

    #trend_frequencies = np.fft.fftfreq(len(time), time[1] - time[0])
    trend_magnitude = np.abs(trend_frequency_domain)

    #periodic_frequencies = np.fft.fftfreq(len(time), time[1] - time[0])
    periodic_magnitude = np.abs(periodic_frequency_domain)

    noise_magnitude = np.abs(noise_frequency_domain)

    trend_peak_freq = np.fft.fftfreq(len(time), time[1] - time[0])[np.argmax(trend_magnitude)]
    periodic_peak_freq = np.fft.fftfreq(len(time), time[1] - time[0])[np.argmax(periodic_magnitude)]
    noise_peak_freq = np.fft.fftfreq(len(time), time[1] - time[0])[np.argmax(noise_magnitude)]

    return trend_peak_freq, periodic_peak_freq, noise_peak_freq

# SSA로 얻은 trend와 periodic에서 주파수 성분을 추출하여 단일값을 리턴하는 함수(total)
def fourier_transform_total(trend, periodic, noise):
    time=np.linspace(0, 10, len(trend)) # 10초 simulation
    trend_frequency_domain = np.fft.fft(trend)
    periodic_frequency_domain = np.fft.fft(periodic)
    noise_frequency_domain = np.fft.fft(noise)

    #trend_frequencies = np.fft.fftfreq(len(time), time[1] - time[0])
    trend_magnitude = np.abs(trend_frequency_domain)

    #periodic_frequencies = np.fft.fftfreq(len(time), time[1] - time[0])
    periodic_magnitude = np.abs(periodic_frequency_domain)
    noise_magnitude = np.abs(noise_frequency_domain)

    trend_spectral_energy = np.sum(trend_magnitude**2)
    periodic_spectral_energy = np.sum(periodic_magnitude**2)
    noise_spectral_energy = np.sum(noise_magnitude**2)

    return trend_spectral_energy, periodic_spectral_energy, noise_spectral_energy

peak

In [5]:
#DDPG
from openpyxl import load_workbook

filename=excel_files[0]

def get_sheetnames_xlsx(filepath):
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    return wb.sheetnames

sheet_names=get_sheetnames_xlsx(excel_dir_path+"/"+filename)

DDPG_list=[]
for i in range(len(sheet_names)):
    df=pd.read_excel(excel_dir_path+"/"+filename, sheet_name=sheet_names[i], engine='openpyxl', index_col=None, header=None)
    
    # NaN 값을 제거한 후 SSA, fourier transform
    compressed_columns = [fourier_transform(do_SSA(df[col].dropna().tolist())[0], do_SSA(df[col].dropna().tolist())[1], do_SSA(df[col].dropna().tolist())[2]) for col in df.columns]
    list_trend = [tup[0] for tup in compressed_columns]
    list_periodic = [tup[1] for tup in compressed_columns]
    list_noise = [tup[2] for tup in compressed_columns]

    DDPG_list.append(list_trend)
    DDPG_list.append(list_periodic)
    DDPG_list.append(list_noise)

In [6]:
#TD3
from openpyxl import load_workbook

filename=excel_files[1]

def get_sheetnames_xlsx(filepath):
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    return wb.sheetnames

sheet_names=get_sheetnames_xlsx(excel_dir_path+"/"+filename)

TD3_list=[]
for i in range(len(sheet_names)):
    df=pd.read_excel(excel_dir_path+"/"+filename, sheet_name=sheet_names[i], engine='openpyxl', index_col=None, header=None)
    
    # NaN 값을 제거한 후 SSA, fourier transform
    compressed_columns = [fourier_transform(do_SSA(df[col].dropna().tolist())[0], do_SSA(df[col].dropna().tolist())[1], do_SSA(df[col].dropna().tolist())[2]) for col in df.columns]
    list_trend = [tup[0] for tup in compressed_columns]
    list_periodic = [tup[1] for tup in compressed_columns]
    list_noise = [tup[2] for tup in compressed_columns]

    TD3_list.append(list_trend)
    TD3_list.append(list_periodic)
    TD3_list.append(list_noise)

In [7]:
final_list=[DDPG_list[i]+TD3_list[i] for i in range(len(TD3_list))]
# (141(feature 개수), 200(simulation 개수)) 3차원인 시계열 데이터에서 2차원 단일값 데이터로 변환

final_array=np.array(final_list)

final_array=final_array.reshape(final_array.shape[1], final_array.shape[0]) #각 instance마다 141개의 feature

final_array.shape


(200, 141)

In [8]:
X=final_array
target=[0]*100+[1]*100
y=np.array(target)

# Multivariate

In [ ]:
from tensorflow import keras
from Model import *
from Utils import *

Data split

Build model

In [10]:
test_model1=build_multivariate_transformer_classification_model( # multivariate transformer model for gait forecasting
    look_back=141, n_features=1,
    num_classes=2,
    head_size=512,
    num_heads=8,
    ff_dim=8,
    num_transformer_blocks=8,
    mlp_units=[512, 256, 64], 
    dropout=0.1,
    mlp_dropout=0.1)

test_model1.compile(loss = {"ts_output" : 'mean_squared_error'},
                                optimizer = keras.optimizers.Adam(learning_rate = 0.001),
                                metrics = {"ts_output" : 'mean_absolute_error'})
callbacks = [keras.callbacks.EarlyStopping(patience = 30, restore_best_weights = True)]

training_history_test_model1 = \
test_model1.fit(
    final_list, 
    y,    
    #validation_data=(XTest, YTest),
    epochs = 200,
    batch_size = 32, 
    callbacks = callbacks,
    verbose = 2)

plotTrainingProgress(training_history=training_history_test_model1, 
                        title="Test model1 multivariate Transformer")
                        


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'numpy.float64\'>"})'}), <class 'numpy.ndarray'>

In [ ]:
training_history_test_model2 = \
test_model1.fit(
    Final, 
    y,    
    #validation_data=(XTest, YTest),
    epochs = 200,
    batch_size = 32, 
    callbacks = callbacks,
    verbose = 2)

plotTrainingProgress(training_history=training_history_test_model2, 
                        title="Test model1 multivariate Transformer")
                        

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).